In [8]:
#Vibha Vijay & Sharaf Tariq

#import packages
import requests
from bs4 import BeautifulSoup


In [9]:
#import packages
import pgeocode

In [10]:
#asking for input
zipcode = input("What is your zipcode?")

What is your zipcode? 22903


In [11]:
#figuring out lat and long based on zipcode
nomi = pgeocode.Nominatim('us')
query = nomi.query_postal_code(zipcode)
lat = "lat=" + str(query["latitude"])
lon = "&lon=" + str(query["longitude"])


print(lat)
print(lon)

lat=38.0339
&lon=-78.4924


In [12]:
#using BeautifulSoup to pull from forecast page 
page = requests.get("https://forecast.weather.gov/MapClick.php?"+ lat + lon)
soup = BeautifulSoup(page.content, 'html.parser')
seven_day = soup.find(id="seven-day-forecast")
forecast_items = seven_day.find_all(class_="tombstone-container")
tonight = forecast_items[0]
print(tonight.prettify())

<div class="tombstone-container">
 <p class="period-name">
  Tonight
  <br/>
  <br/>
 </p>
 <p>
  <img alt="Tonight: A chance of rain after 1am.  Increasing clouds, with a low around 39. South wind 5 to 8 mph, with gusts as high as 18 mph.  Chance of precipitation is 50%." class="forecast-icon" src="DualImage.php?i=nbkn&amp;j=nra&amp;jp=50" title="Tonight: A chance of rain after 1am.  Increasing clouds, with a low around 39. South wind 5 to 8 mph, with gusts as high as 18 mph.  Chance of precipitation is 50%."/>
 </p>
 <p class="short-desc">
  Mostly Cloudy
  <br/>
  then Chance
  <br/>
  Rain
 </p>
 <p class="temp temp-low">
  Low: 39 °F
 </p>
</div>


In [14]:
#pulling data for seven day forecast from specific HTML containers on weather site
period_tags = seven_day.select(".tombstone-container .period-name")
periods = [pt.get_text() for pt in period_tags]
periods

short_descs = [sd.get_text() for sd in seven_day.select(".tombstone-container .short-desc")]
temps = [t.get_text() for t in seven_day.select(".tombstone-container .temp")]
descs = [d["title"] for d in seven_day.select(".tombstone-container img")]
print(short_descs)
print(temps)
print(descs)

['Mostly Cloudythen ChanceRain', 'Rain', 'Partly Cloudy', 'Mostly Sunny', 'Mostly Clear', 'Mostly Sunny', 'Chance Rain', 'Chance Rain', 'Chance Rain']
['Low: 39 °F', 'High: 62 °F', 'Low: 32 °F', 'High: 48 °F', 'Low: 28 °F', 'High: 52 °F', 'Low: 37 °F', 'High: 54 °F', 'Low: 45 °F']
['Tonight: A chance of rain after 1am.  Increasing clouds, with a low around 39. South wind 5 to 8 mph, with gusts as high as 18 mph.  Chance of precipitation is 50%.', 'Saturday: Rain, mainly before 1pm.  High near 62. Southwest wind 11 to 14 mph, with gusts as high as 26 mph.  Chance of precipitation is 80%. New precipitation amounts of less than a tenth of an inch possible. ', 'Saturday Night: Partly cloudy, with a low around 32. West wind 5 to 7 mph becoming calm  after midnight. ', 'Sunday: Mostly sunny, with a high near 48. Calm wind becoming south around 6 mph in the afternoon. ', 'Sunday Night: Mostly clear, with a low around 28. Light and variable wind. ', 'Monday: Mostly sunny, with a high near 52.'

In [15]:
#creating 7 day forecast dataframe
import pandas as pd
weather = pd.DataFrame({
    "period": periods,
    "short_desc": short_descs,
    "temp": temps,
    "desc":descs
})
weather

,period,short_desc,temp,desc
0,Tonight,Mostly Cloudythen ChanceRain,Low: 39 °F,Tonight: A chance of rain after 1am. Increasi...
1,Saturday,Rain,High: 62 °F,"Saturday: Rain, mainly before 1pm. High near ..."
2,SaturdayNight,Partly Cloudy,Low: 32 °F,"Saturday Night: Partly cloudy, with a low arou..."
3,Sunday,Mostly Sunny,High: 48 °F,"Sunday: Mostly sunny, with a high near 48. Cal..."
4,SundayNight,Mostly Clear,Low: 28 °F,"Sunday Night: Mostly clear, with a low around ..."
5,Monday,Mostly Sunny,High: 52 °F,"Monday: Mostly sunny, with a high near 52."
6,MondayNight,Chance Rain,Low: 37 °F,Monday Night: A chance of rain. Mostly cloudy...
7,Tuesday,Chance Rain,High: 54 °F,"Tuesday: A chance of rain. Mostly cloudy, wit..."
8,TuesdayNight,Chance Rain,Low: 45 °F,Tuesday Night: A chance of rain. Mostly cloud...


In [16]:
#pulling current conditions from HTML container on weather site 
extra_day = soup.find(id="current_conditions_detail")
extra_items = extra_day.find_all(class_="text-right")
extra_tonight = extra_items[0]
print(extra_tonight.prettify())


<td class="text-right">
 <b>
  Humidity
 </b>
</td>



In [17]:
#consolidating current condition data 
hum = (soup.find_all('b')[4].get_text())+ ":" + soup.find_all('td')[1].get_text()
wind = soup.find_all('b')[5].get_text()+ ":" + soup.find_all('td')[3].get_text()
dew = soup.find_all('b')[7].get_text()+ ":" + soup.find_all('td')[7].get_text()
last = soup.find_all('b')[9].get_text()+ ":" + soup.find_all('td')[11].get_text()

print(hum)
print(wind)
print(dew)
print(last)

# importing pandas as pd
import pandas as pd
 
# import the StrinIO function
# from io module
from io import StringIO
 
# wrap the string data in StringIO function
StringData = StringIO("""Humidity;Wind_Speed;Dewpoint;Wind_Chill
    38%;S 7mph;22°F (-6°C);42°F (6°C)
     """)
 
# let's read the data using the Pandas
# read_csv() function
current = pd.read_csv(StringData, sep =";")
 
# Print the dataframe
print(current)


Humidity:36%
Wind Speed:S 7 mph
Dewpoint:23°F (-5°C)
Wind Chill:46°F (8°C)
  Humidity Wind_Speed     Dewpoint  Wind_Chill
0      38%     S 7mph  22°F (-6°C)  42°F (6°C)


In [18]:
#importing packages for MongoDB
import pandas as pd
import pymongo
from pymongo import MongoClient

In [19]:
# Making a Connection with MongoClient
client = MongoClient("mongodb://localhost:27017/")
# database
db = client["admin"]
# collection
lab05= db["Lab05"]

In [20]:
#Dumping weather into Mongo
weather.reset_index(inplace=True)
data_dict = weather.to_dict("records")
lab05.insert_one({"index":"Weather","weather":data_dict})

In [21]:
#Dumping current conditions into Mongo
current.reset_index(inplace=True)
data_dict = current.to_dict("records")
lab05.insert_one({"index":"Current Conditions","current":data_dict})